## Linear Program with Storage

This linear program builds on the emissions_costs script by incorporating the hourly state of charge and corresponding constraints for storage resources.

In [100]:
%reload_ext autoreload
%autoreload 2

import numpy as np # numerical library
import matplotlib.pyplot as plt # plotting library
import datetime as dt
import pandas as pd

from ortools.linear_solver import pywraplp

In [101]:
import utils

In [102]:
#Changeable parameters.

battery_duration = 4
initial_state_of_charge = 0

In [103]:
storage_solver = pywraplp.Solver('HarborOptimization',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

#Introduce objective object so we can refer to it in the for loop.
objective = storage_solver.Objective()

In [104]:
# Load Harbor historical hourly generation and emissions for 2014-2018. Group by datetime to sum generation and emissions from all units. Filter for specific year -- make sure selected year isn't leap year.
harborgen = utils.get_harbor_data('data/HarborHourly_2014-18.csv')
harborgen = harborgen.groupby(['datetime'])['mwh'].sum()

# Create string object that is the chosen year
year = '2018'

# Filter Harbor data by the chosen year, using the string object above.
harborgen = harborgen.filter(like=year, axis=0)

In [105]:
# Load generation profiles for nondispatchable resources (KWh generated each hour by 1 KW of capacity).
profiles = pd.read_csv('data/gen_profiles.csv')

# Match profiles index to harborgen index. **Currently this only works for one year -- for multiple years, need to repeat gen profiles.
profiles.insert(0, 'datetime', harborgen.index)
profiles = profiles.set_index('datetime')

In [106]:
profiles.head()

,solar,ee_1,ee_2,ee_3,ee_4,ee_5,ee_6,ee_7,ee_8,ee_9,ee_10
datetime,,,,,,,,,,,
2018-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
harborgen.head()

datetime
2018-01-01 00:00:00    0.0
2018-01-01 01:00:00    0.0
2018-01-01 02:00:00    0.0
2018-01-01 03:00:00    0.0
2018-01-01 04:00:00    0.0
Name: mwh, dtype: float64

In [108]:
resources = pd.read_csv('data/resource_costs.csv')
resources = resources.set_index('resource')
resources

,legacy,existing_mw,dispatchable,capex,fixed,variable,CO2,NOX,SO2,PM2.5,PM10,CH4,TOTAL/MWH
resource,,,,,,,,,,,,,
gas_harbor,y,450.0,y,NaN,NaN,0.21,NaN,NaN,NaN,NaN,NaN,NaN,0.36
outofbasin,y,3000.0,y,NaN,NaN,0.21,NaN,NaN,NaN,NaN,NaN,NaN,0.36
gas_repower,n,NaN,y,5.0,NaN,0.10,0.06,0.06,0.06,0.06,0.06,0.06,0.36
solar,n,NaN,n,6.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
demand_response,n,NaN,y,4.0,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,0.00
ee_1,n,NaN,n,5.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ee_2,n,NaN,n,4.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ee_3,n,NaN,n,3.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ee_4,n,NaN,n,2.0,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [109]:
storage = pd.read_csv('data/storage_costs.csv')
storage = storage.set_index('resource')
storage

,capex ($/MW),fixed ($/MW-year),variable($/MWh),efficiency
resource,,,,
storage_utility,1287000.0,2750.0,88.0,0.9
storage_ci,1287000.0,2750.0,88.0,0.9
storage_res,1287000.0,2750.0,88.0,0.9
storage_diesel,1287000.0,2750.0,88.0,0.9


In [110]:
outofbasin_emissions = pd.read_csv('data/outofbasin_emissions.csv')
outofbasin_emissions.insert(0, 'datetime', harborgen.index)
outofbasin_emissions = outofbasin_emissions.set_index('datetime')
outofbasin_emissions

,CO2,NOX,SO2,PM2.5,PM10,CH4,TOTAL/MWH
datetime,,,,,,,
2018-01-01 00:00:00,1,2,1.5,2,0,2,8.5
2018-01-01 01:00:00,1,2,1.5,2,0,2,8.5
2018-01-01 02:00:00,1,2,1.5,2,0,2,8.5
2018-01-01 03:00:00,1,2,1.5,2,0,2,8.5
2018-01-01 04:00:00,1,2,1.5,2,0,2,8.5
...,...,...,...,...,...,...,...
2018-12-31 19:00:00,1,2,1.5,2,0,2,8.5
2018-12-31 20:00:00,1,2,1.5,2,0,2,8.5
2018-12-31 21:00:00,1,2,1.5,2,0,2,8.5


In [111]:
#Hourly monetized grid emissions for the whole LADWP grid will be used to quantify the emissions impacts of storage charging.
whole_grid_emissions = pd.read_csv('data/whole_grid_emissions.csv')
whole_grid_emissions.insert(0, 'datetime', harborgen.index)
whole_grid_emissions = whole_grid_emissions.set_index('datetime')
whole_grid_emissions

,CO2,NOX,SO2,PM2.5,PM10,CH4,TOTAL/MWH
datetime,,,,,,,
2018-01-01 00:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 01:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 02:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 03:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-01-01 04:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
...,...,...,...,...,...,...,...
2018-12-31 19:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-12-31 20:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5
2018-12-31 21:00:00,1.0,2.0,1.5,2.0,0.0,2.0,8.5


In [112]:
# Declare nameplate capacity variables for each resource in resource cost and storage cost dataframes.
capacity_vars = {}
for resource in resources.index:
    if resources.loc[str(resource)]['legacy'] == 'n':
        capacity = storage_solver.NumVar(0, storage_solver.infinity(), str(resource))
        capacity_vars[resource] = capacity

for resource in storage.index:
    capacity = storage_solver.NumVar(0, storage_solver.infinity(), str(resource))
    capacity_vars[resource] = capacity
    
capacity_vars

{'gas_repower': gas_repower,
 'solar': solar,
 'demand_response': demand_response,
 'ee_1': ee_1,
 'ee_2': ee_2,
 'ee_3': ee_3,
 'ee_4': ee_4,
 'ee_5': ee_5,
 'ee_6': ee_6,
 'ee_7': ee_7,
 'ee_8': ee_8,
 'ee_9': ee_9,
 'ee_10': ee_10,
 'storage_utility': storage_utility,
 'storage_ci': storage_ci,
 'storage_res': storage_res,
 'storage_diesel': storage_diesel}

In [113]:
#Create filtered dataframes for dispatchable and nondispatchable resources.
disp = resources.loc[resources['dispatchable'] == 'y']
nondisp = resources.loc[resources['dispatchable'] == 'n']
disp

,legacy,existing_mw,dispatchable,capex,fixed,variable,CO2,NOX,SO2,PM2.5,PM10,CH4,TOTAL/MWH
resource,,,,,,,,,,,,,
gas_harbor,y,450.0,y,NaN,NaN,0.21,NaN,NaN,NaN,NaN,NaN,NaN,0.36
outofbasin,y,3000.0,y,NaN,NaN,0.21,NaN,NaN,NaN,NaN,NaN,NaN,0.36
gas_repower,n,NaN,y,5.0,NaN,0.10,0.06,0.06,0.06,0.06,0.06,0.06,0.36
demand_response,n,NaN,y,4.0,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [114]:
#Create a dictionary to hold a list for each dispatchable resource that keeps track of its hourly generation variables.
disp_gen = {}
for resource in disp.index:
    disp_gen[resource] = []

In [115]:
disp_gen

{'gas_harbor': [], 'outofbasin': [], 'gas_repower': [], 'demand_response': []}

In [116]:
#Create a dictionary to hold a list for each storage resource that keeps track of its hourly charge variables.
charge_vars = {}
for resource in storage.index:
    charge_vars[resource] = []

In [117]:
#Create a dictionary to hold a list for each storage resource that keeps track of its hourly discharge variables.
discharge_vars = {}
for resource in storage.index:
    discharge_vars[resource] = []

In [118]:
#Creates a dictionary to track the hourly state of charge of each storage resource. Each value represents the state of charge at the start of each timestep.
state_of_charge_vars = {}
for resource in storage.index:
    state_of_charge_vars[resource] = [] 

In [119]:
#Loop through every hour, creating 1) hourly generation variables for each dispatchable resource, 2) hourly constraints, and 3) adding variable cost coefficients to each hourly generation variable.
for ind in harborgen.index:
    
    #Summed generation from all resources must be equal or greater to demand in all hours.
    fulfill_demand = storage_solver.Constraint(harborgen.loc[ind], storage_solver.infinity())
    
    #Create hourly charge and discharge variables for each storage resource and store in respective dictionaries. 
    for resource in storage.index:
        
        #Create hourly charge and discharge variables for each storage resource.
        charge = storage_solver.NumVar(0, storage_solver.infinity(), '_charge'+ str(ind))
        discharge = storage_solver.NumVar(0, storage_solver.infinity(), '_discharge'+ str(ind))
        
        #Add variable cost of charging and monetized emissions to objective function.
        variable_cost = whole_grid_emissions.loc[ind,'TOTAL/MWH']+ storage.loc[resource,'variable($/MWh)']
        objective.SetCoefficient(charge, variable_cost)
        
        #Limit hourly charge and discharge variables to storage max power (MW).
        max_charge = storage_solver.Constraint(0, storage_solver.infinity())
        max_charge.SetCoefficient(capacity_vars[resource], 1)
        max_charge.SetCoefficient(charge, -1)
        
        max_discharge = storage_solver.Constraint(0, storage_solver.infinity())
        max_discharge.SetCoefficient(capacity_vars[resource], 1)
        max_discharge.SetCoefficient(discharge, -1)
        
        #Keep track of hourly charge and discharge variables by appending to lists for each storage resource.
        charge_vars[resource].append(charge)
        discharge_vars[resource].append(discharge)
        
        #Hourly discharge variables of storage resources are incorporated into the fulfill demand constraint. If storage can only charge from portfolio resources, include the charge variable in this constraint.
        efficiency = storage.loc[resource, 'efficiency']
        fulfill_demand.SetCoefficient(discharge, efficiency)
        #Include the line below if storage can only charge from portfolio resources.
        #fulfill_demand.SetCoefficient(charge, -1)
        
        #Creates hourly state of charge variable, representing the state of charge at the end of each timestep. 
        state_of_charge = storage_solver.NumVar(0, storage_solver.infinity(), 'state_of_charge'+ str(ind))
        
        #Temporal coupling of storage state of charge.
        if harborgen.index.get_loc(ind) > 0:
            temporal = storage_solver.Constraint(0, 0)
            temporal.SetCoefficient(state_of_charge, -1)
            temporal.SetCoefficient(discharge, -1)
            temporal.SetCoefficient(charge, efficiency)
            #Get the state of charge from previous timestep to include in the temporal coupling constraint.
            previous_state = state_of_charge_vars[resource][-1]
            temporal.SetCoefficient(previous_state, 1)
        else: 
            temporal = storage_solver.Constraint(initial_state_of_charge, initial_state_of_charge)
            temporal.SetCoefficient(state_of_charge, 1)
            temporal.SetCoefficient(discharge, 1)
            temporal.SetCoefficient(charge, -efficiency)
        
        #Add hourly state of charge variable to corresponding list for each storage resource.
        state_of_charge_vars[resource].append(state_of_charge)
        
        #Creates constraint setting max for storage state of charge.
        #*** TO-DO: Alter this to instead pull "battery_duration" from battery duration column in storage resource csv (to accomodate different durations)
        max_storage = storage_solver.Constraint(0, storage_solver.infinity())
        max_storage.SetCoefficient(state_of_charge, -1)
        max_storage.SetCoefficient(capacity_vars[resource], battery_duration)
        
        #Creates constraint ensuring that no net energy is supplied by storage (ending state of charge is equal to initial state of charge).
        if harborgen.index.get_loc(ind) == len(harborgen)-1:
            ending_state = storage_solver.Constraint(initial_state_of_charge, initial_state_of_charge)
            ending_state.SetCoefficient(state_of_charge, 1)
            

    #Create generation variable for each dispatchable resource for every hour. Append hourly gen variable to the list for that resource, located in the disp_gen dictionary.
    #Create constraint that generation must be less than or equal to capacity for each dispatchable resource for all hours.
    for resource in disp.index:
        
        gen = storage_solver.NumVar(0, storage_solver.infinity(), '_gen'+ str(ind))
        disp_gen[resource].append(gen)
        if resource == 'outofbasin':
            # TODO: Incorporate transmission cost into variable cost for outofbasin option.
            variable_cost = outofbasin_emissions.loc[ind,'TOTAL/MWH']+ disp.loc[resource,'variable']
            objective.SetCoefficient(gen, variable_cost)
        else:
            variable_cost = disp.loc[resource,'TOTAL/MWH']+ disp.loc[resource,'variable']
            objective.SetCoefficient(gen, variable_cost)
        
        #Set coefficients for the hourly gen variables for the fulfill_demand constraint.
        fulfill_demand.SetCoefficient(gen, 1)
        
        #Set coefficients for dispatchable capacity variables and hourly gen variables for the max_gen = capacity constraint. 
        #For legacy resources, contrains maximum hourly generation to existing capacity.
        if resources.loc[str(resource)]['legacy'] == 'n':
            max_gen = storage_solver.Constraint(0, storage_solver.infinity())
            capacity = capacity_vars[resource]
            max_gen.SetCoefficient(capacity, 1)
            max_gen.SetCoefficient(gen, -1)
        else:
            capacity = resources.loc[str(resource)]['existing_mw']
            max_gen = storage_solver.Constraint(0, capacity)
            max_gen.SetCoefficient(gen, 1)
    
    #For each nondispatchable resource, set the coefficient of the capacity variable to its generation profile scaling factor. **Make sure units are aligned here (kw vs. mw capacities)
    for resource in nondisp.index: 
        capacity = capacity_vars[resource]
        coefficient = profiles.loc[ind, str(resource)]
        fulfill_demand.SetCoefficient(capacity, coefficient)


In [120]:
## Add in fixed costs -- we need to assume a certain plant lifetime and amortize over that period, using discount rate. ******
for resource in resources.index:
    if resources.loc[str(resource)]['legacy'] == 'n':
        capex = resources.loc[resource, 'capex']
        objective.SetCoefficient(capacity_vars[resource], capex)
        
for resource in storage.index:
    capex = storage.loc[resource, 'capex ($/MW)']
    objective.SetCoefficient(capacity_vars[resource], capex)

objective.SetMinimization()
status = storage_solver.Solve()
if status == storage_solver.OPTIMAL:
    print("Solver found optimal solution.")
    print("total cost =", objective.Value())

    for resource in capacity_vars:
        print(str(capacity_vars[resource]) + ' capacity =' + str(capacity_vars[resource].solution_value()))

    # Sum generation for each resource and print.
    for resource in disp.index:
        summed_gen = 0
        for i_gen in disp_gen[str(resource)]:
            summed_gen += i_gen.solution_value()
        print(str(resource) + ': annual generation =' + str(summed_gen))

    ## Sum annual generation for nondispatchable resources.
    summed_gen = profiles.sum()

    for resource in nondisp.index:
        summed_gen.filter(resource)
        capacity = capacity_vars[resource].solution_value()
        gen = summed_gen[resource] * capacity
        print(str(resource) + ': annual generation =' + str(gen))
else:
    print("Solver exited with error code {}".format(status))

Solver found optimal solution.
total cost = 4153.065666666666
gas_repower capacity =0.0
solar capacity =0.0
demand_response capacity =216.0
ee_1 capacity =0.0
ee_2 capacity =0.0
ee_3 capacity =0.0
ee_4 capacity =0.0
ee_5 capacity =716.6666666666666
ee_6 capacity =0.0
ee_7 capacity =0.0
ee_8 capacity =0.0
ee_9 capacity =0.0
ee_10 capacity =0.0
storage_utility capacity =0.0
storage_ci capacity =0.0
storage_res capacity =0.0
storage_diesel capacity =0.0
gas_harbor: annual generation =21.000000000000053
outofbasin: annual generation =0.0
gas_repower: annual generation =0.0
demand_response: annual generation =17069.526666666672
solar: annual generation =0.0
ee_1: annual generation =0.0
ee_2: annual generation =0.0
ee_3: annual generation =0.0
ee_4: annual generation =0.0
ee_5: annual generation =1244154.8333333281
ee_6: annual generation =0.0
ee_7: annual generation =0.0
ee_8: annual generation =0.0
ee_9: annual generation =0.0
ee_10: annual generation =0.0
